# 🚀 Tam Otomatik JetX Model Eğitmeni (v3.2)

Bu not defteri, JetX tahmin sistemi için yeniden yapılandırılmış ve tam otomatik bir eğitim akışı sağlar.

## ✨ Yeni Özellikler (v3.2)

- **ngrok Ön Kontrolü:** `ngrok` tüneli oluşturulmadan önce bağlantı ve `authtoken` test edilir, olası hatalar en başta tespit edilir.
- **Otomatik Hiperparametre Optimizasyonu (HPO):** `Optuna` entegrasyonu ile her model için en iyi hiperparametreler otomatik olarak bulunur.
- **Deney Takibi:** `MLflow` entegrasyonu ile tüm eğitim süreçleri, metrikler ve modeller kaydedilir ve izlenebilir.
- **Merkezi Yapılandırma:** Tüm ayarlar artık proje kök dizinindeki `config.yaml` dosyasından yönetilir.

**⚠️ Başlamadan Önce: Runtime > Change runtime type > GPU seçeneğini etkinleştirin!**

## 🛠️ 1. Sistem Kurulumu ve Projeyi Yükleme

In [ ]:
import os
import sys
import subprocess

def setup_environment():
    """Clones or updates the repository and sets up the environment."""
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"

    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)

    # Add src to Python path
    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")

    # Install dependencies
    requirements_path = os.path.join(project_dir, 'requirements_enhanced.txt')
    if os.path.exists(requirements_path):
        print("📦 Gerekli kütüphaneler yükleniyor...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", requirements_path])
        print("✅ Kütüphaneler başarıyla yüklendi.")

setup_environment()
print("🎉 Kurulum tamamlandı!")

## 📊 2. MLflow Deney Takibini Başlatma

Bu hücre, MLflow arayüzünü başlatır ve `ngrok` kullanarak size erişebileceğiniz bir public URL verir. **Lütfen aşağıdaki hücreye ngrok authtoken'ınızı girin.**

1. [ngrok Dashboard](https://dashboard.ngrok.com/get-started/your-authtoken) adresine gidin.
2. Authtoken'ınızı kopyalayın.
3. Aşağıdaki `NGROK_AUTH_TOKEN` değişkenine yapıştırın.

In [ ]:
from getpass import getpass
from colab.health_check import ColabHealthCheck
import time

# ngrok authtoken'ınızı buraya girin
NGROK_AUTH_TOKEN = getpass('Lütfen ngrok authtokeninizi girin: ')

# ngrok bağlantısını önceden test et
health_checker = ColabHealthCheck()
ngrok_ok, error_message = health_checker.check_ngrok_tunnel(NGROK_AUTH_TOKEN)

if ngrok_ok:
    print("✅ ngrok bağlantı testi başarılı.")
    
    # MLflow UI'ı arka planda başlat
    get_ipython().system_raw('mlflow ui --host 0.0.0.0 &')
    
    # Give MLflow a moment to start
    time.sleep(5)
    
    print("\n✅ MLflow arayüzü için public URL oluşturuluyor...")
    print("👇 Aşağıdaki linke tıklayarak deneyleri izleyebilirsiniz:")
    
    try:
        from pyngrok import ngrok
        # Connect to the MLflow UI port and print only the public URL
        public_url = ngrok.connect(5000, "http").public_url
        print(f"--> {public_url}")
    except Exception as e:
        print(f"❌ ngrok tüneli oluşturulamadı. Hata: {e}")
else:
    print(f"\n❌ ngrok ön kontrolü başarısız oldu. Lütfen authtoken'ınızı ve internet bağlantınızı kontrol edin.")
    print(f"   Hata Detayı: {error_message}")

## 🚀 3. Otomatik Eğitim Sürecini Başlatma

In [ ]:
import torch
from src.training.master_trainer import MasterTrainer
import traceback

# Cihaz tespiti (GPU veya CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    print(f"✅ GPU bulundu: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU bulunamadı. Eğitim CPU ile devam edecek.")

try:
    # MasterTrainer'ı başlat ve tüm eğitim sürecini çalıştır.
    master_trainer = MasterTrainer(models_to_train=['N-Beats', 'TFT', 'LSTM'], device=device)
    master_trainer.run()
except Exception as e:
    print(f"❌ Eğitim sırasında beklenmedik bir hata oluştu: {e}")
    traceback.print_exc()